# Week 13. Class imbalanced problem

과제로 아래 cell을 작성해주세요

주석의 경우 이미지, 테이블 등의 표현이 어려운 관계로 받지 않겠습니다

데이터는 실습과 같은 데이터를 사용합니다

### 실습 1. Class imbalanced problem

클래스 불균형 문제를 해결하기 위한 본인의 방법을 제안해주세요

간단하게 sampling하셔도 좋고 feature들을 줄이셔도 좋습니다

다양한 방법으로 training set을 전처리 한 뒤 결과로 정리해주세요

In [1]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = ""

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,train_test_split

from plotly import express as px
from plotly import graph_objs as go
tf.config.get_visible_devices()

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.decomposition import PCA

In [7]:
train = pd.read_csv('12_train.csv')
test = pd.read_csv('12_test.csv')

In [8]:
train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,10000,2,2,2,21,2,2,3,2,0,...,9768,8430,20735,3905,0,0,300,2165,2666,1
1,400000,2,1,2,30,0,0,0,0,0,...,191514,121092,166173,26876,10134,85450,10140,88097,10086,0
2,260000,2,2,1,30,0,0,0,0,0,...,155493,158624,161902,7129,5179,5276,4537,4670,4717,0
3,50000,1,2,2,23,0,0,-1,-1,-2,...,0,0,0,434,780,0,0,0,0,0
4,60000,2,3,1,29,0,0,0,0,0,...,34416,28818,28546,2200,2300,3000,1100,1000,1000,0


In [9]:
del train['EDUCATION']
del train['MARRIAGE']
del train['SEX']

In [12]:
del test['EDUCATION']
del test['MARRIAGE']
del test['SEX']

In [14]:
test.head()

,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,150000,34,-2,-2,-2,-2,-2,-2,1198,10848,...,16864,17199,20025,12003,5000,16864,18000,20025,51514,0
1,200000,35,1,2,0,0,0,0,205002,195441,...,304997,188504,189527,41,8597,8583,6880,7235,6839,1
2,80000,26,1,-2,-2,-2,-2,-2,0,0,...,0,0,0,0,3100,0,0,0,0,0
3,10000,37,0,0,0,0,0,-2,14498,7512,...,5275,0,0,2248,1120,375,0,0,0,0
4,50000,24,1,2,2,0,0,2,50789,52158,...,50959,51121,49344,2500,0,2100,3500,0,2000,0


In [13]:
standard_scaler = StandardScaler()

tr_x, tr_y = train.iloc[:,:-1], train.iloc[:,-1]
ts_x, ts_y = test.iloc[:,:-1], test.iloc[:,-1]

columns = tr_x.columns
tr_x = standard_scaler.fit_transform(tr_x)
ts_x = standard_scaler.transform(ts_x)

In [16]:
from imblearn.over_sampling import SMOTE

In [19]:
smote=SMOTE(n_jobs=-1)
scores=pd.DataFrame()

In [20]:
smote_tr_x, smote_tr_y = smote.fit_resample(tr_x,tr_y)

print(f'Class 별 instance 수 0 : {(smote_tr_y==0).sum()}, 1 :{(smote_tr_y==1).sum()}')
print('Class 비율 : ', (smote_tr_y==1).sum()/(smote_tr_y==0).sum())

smote_models=[(j,i) for i,j in zip([AdaBoostClassifier(DecisionTreeClassifier(),),
                                 RandomForestClassifier(),
                                 DecisionTreeClassifier(),
                                 KNeighborsClassifier(),
                                     ],
                                ['AdaBoost','RF','DT','kNN'])]

for name, model in smote_models:
    model.fit(smote_tr_x, smote_tr_y)
    print(f'SMOTE {name} : {model.score(ts_x, ts_y):.5}%')
    scores.loc['SMOTE', name]=model.score(ts_x, ts_y)
    

Class 별 instance 수 0 : 17523, 1 :17523
Class 비율 :  1.0
SMOTE AdaBoost : 0.7696%
SMOTE RF : 0.79547%
SMOTE DT : 0.69773%
SMOTE kNN : 0.6596%


In [21]:
del train['PAY_0']
del test['PAY_0']
del train['PAY_2']
del test['PAY_2']

In [22]:
standard_scaler = StandardScaler()

tr_x, tr_y = train.iloc[:,:-1], train.iloc[:,-1]
ts_x, ts_y = test.iloc[:,:-1], test.iloc[:,-1]

columns = tr_x.columns
tr_x = standard_scaler.fit_transform(tr_x)
ts_x = standard_scaler.transform(ts_x)

In [23]:
smote_tr_x, smote_tr_y = smote.fit_resample(tr_x,tr_y)

print(f'Class 별 instance 수 0 : {(smote_tr_y==0).sum()}, 1 :{(smote_tr_y==1).sum()}')
print('Class 비율 : ', (smote_tr_y==1).sum()/(smote_tr_y==0).sum())

smote_models=[(j,i) for i,j in zip([AdaBoostClassifier(DecisionTreeClassifier(),),
                                 RandomForestClassifier(),
                                 DecisionTreeClassifier(),
                                 KNeighborsClassifier(),
                                     ],
                                ['AdaBoost','RF','DT','kNN'])]

for name, model in smote_models:
    model.fit(smote_tr_x, smote_tr_y)
    print(f'SMOTE {name} : {model.score(ts_x, ts_y):.5}%')
    scores.loc['SMOTE', name]=model.score(ts_x, ts_y)
    

Class 별 instance 수 0 : 17523, 1 :17523
Class 비율 :  1.0
SMOTE AdaBoost : 0.7492%
SMOTE RF : 0.77133%
SMOTE DT : 0.684%
SMOTE kNN : 0.63787%


In [24]:
from imblearn.under_sampling import RandomUnderSampler
rus=RandomUnderSampler()

In [25]:
rus_tr_x, rus_tr_y = rus.fit_resample(tr_x,tr_y)

print(f'Class 별 instance 수 0 : {(rus_tr_y==0).sum()}, 1 :{(rus_tr_y==1).sum()}')
print('Class 비율 : ', (rus_tr_y==1).sum()/(rus_tr_y==0).sum())

rus_models=[(j,i) for i,j in zip([AdaBoostClassifier(DecisionTreeClassifier(),),
                                 RandomForestClassifier(),
                                 DecisionTreeClassifier(),
                                 KNeighborsClassifier(),
                                     ],
                                ['AdaBoost','RF','DT','kNN'])]

for name, model in rus_models:
    model.fit(rus_tr_x, rus_tr_y)
    print(f'Random under sampling {name} : {model.score(ts_x, ts_y):.5}%')
    scores.loc['Random US', name]=model.score(ts_x, ts_y)
    

Class 별 instance 수 0 : 4977, 1 :4977
Class 비율 :  1.0
Random under sampling AdaBoost : 0.62973%
Random under sampling RF : 0.69027%
Random under sampling DT : 0.58707%
Random under sampling kNN : 0.6232%


데이터셋의 특징을 줄이고 oversampling중 smote방법을 통해서 클래스 불균형 문제를 해결할려고 시도를 해보았지만  
특징을 줄일수록 성능이 더 나빠지는것을 볼 수 있었다. oversampling이 아닌 under sampling으로 시도를 해보아도 특징을 줄이기 전보다  
성능이 더 안좋아 진 것을 결과로 확인할 수 있었다.   
만약에 데이터 특징별로 중요도를 알아내서 중요도가 낮은 순서대로 특징을 제거하면 좋은 성능을 얻을 거라고 예측합니다.
